In [1]:
!pip install spacy
!pip install sqlglot
!pip install sqlparse
!pip install accelerate
!pip install pandas sqlalchemy
!pip install -U sentence-transformers
!pip install -i https://pypi.org/simple/ bitsandbytes


%env http_proxy=http://http.proxy.aws.fmrcloud.com:8000/
%env https_proxy=http://http.proxy.aws.fmrcloud.com:8000/
!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_trf

/bin/bash: /home/dsp/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://:****@artifactory.fmr.com/api/pypi/pypi-prereleases/simple
/bin/bash: /home/dsp/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://:****@artifactory.fmr.com/api/pypi/pypi-prereleases/simple
/bin/bash: /home/dsp/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://:****@artifactory.fmr.com/api/pypi/pypi-prereleases/simple
/bin/bash: /home/dsp/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://:****@artifactory.fmr.com/api/pypi/pypi-prereleases/simple
/bin/bash: /home/dsp/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://:****@artifactory.fmr.com/api/pypi/pypi-prereleases/simple
/bin/bash: /home/dsp/mini

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 457.4 MB 10 kB/s /s eta 0:00:01███████ | 442.0 MB 159.4 MB/s eta 0:00:01�████████████████▍| 448.3 MB 159.4 MB/s eta 0:00:01�████████████████▊| 453.2 MB 159.4 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [2]:
import os
import numpy as np
import pandas as pd

## Chat with Tabular Data

This notebook contains code to demonstrate NL2SQL capabilities of modern LLMs and only uses open-source data and libraries

### Download input data

We will be using the Churn Modelling dataset from Kaggle. Please note that a Kaggle account is needed to download the dataset.

This data set contains details of a bank's customers and the target variable is a binary variable reflecting the fact whether the customer left the bank (closed his account) or he continues to be a customer.

In [3]:
input_filepath = 'Churn_Modelling.csv'
table_name = input_filepath.rsplit('.', maxsplit=1)[0]

df = pd.read_csv(input_filepath)
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


### Load CSV data into a SQLite DB

Since our LLM will generate SQL code, we must load our CSV data into a SQL compatible database.  
We also use the DB engine to create a DDL or description of the table which will be used as part of the LLM prompt.

In [4]:
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///mysqlitedb.db")

try:
    df.to_sql(table_name, engine, index=False)
except Exception as e:
    print(f'Error: {e}')
    print('')

with engine.connect() as conn, conn.begin():
    sqlite_master = pd.read_sql_query("SELECT * FROM sqlite_master", conn)
sqlite_master['sql_fmt'] = sqlite_master['sql'].apply(lambda z: [x.strip().strip(',').rsplit(' ', maxsplit=1) for x in z.split('\n')[1:-1]])

table_desc_dict = {}
for _, row in sqlite_master.iterrows():
    table_desc_dict[row['name']] = row['sql']

schema = table_desc_dict[table_name]
print(schema)

Error: Table 'Churn_Modelling' already exists.

CREATE TABLE "Churn_Modelling" (
	"RowNumber" BIGINT, 
	"CustomerId" BIGINT, 
	"Surname" TEXT, 
	"CreditScore" BIGINT, 
	"Geography" TEXT, 
	"Gender" TEXT, 
	"Age" BIGINT, 
	"Tenure" BIGINT, 
	"Balance" FLOAT, 
	"NumOfProducts" BIGINT, 
	"HasCrCard" BIGINT, 
	"IsActiveMember" BIGINT, 
	"EstimatedSalary" FLOAT, 
	"Exited" BIGINT
)


In [5]:
table_desc_df = {}
for _, row in sqlite_master.iterrows():
    table_desc_df[row['name']] = pd.DataFrame(columns=['name', 'type'], data=row['sql_fmt'])
    table_desc_df[row['name']]['comment'] = np.nan

table_desc_df['Churn_Modelling']

,name,type,comment
0,"""RowNumber""",BIGINT,NaN
1,"""CustomerId""",BIGINT,NaN
2,"""Surname""",TEXT,NaN
3,"""CreditScore""",BIGINT,NaN
4,"""Geography""",TEXT,NaN
5,"""Gender""",TEXT,NaN
6,"""Age""",BIGINT,NaN
7,"""Tenure""",BIGINT,NaN
8,"""Balance""",FLOAT,NaN
9,"""NumOfProducts""",BIGINT,NaN


### DDL Pruning and Prompt Generation

Tables can have columns numbering in the hundreds and thousands. To answer a given query, it is not necessary for all the column names to be included in the LLM prompt.  
"Pruning" the table DDL before creating the prompt makes it easier for the LLM to generate valid SQL output and prevents hallucinations.  
We do this by using semantic similarity between the user query and the various column names. 

In [7]:
import spacy
import torch
import sqlglot
import numpy as np
import os, re, logging, pickle
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

/home/dsp/miniconda3/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [38]:
def knn_(query: str, all_embs: torch.tensor, top_k: int, threshold: float) -> tuple[torch.tensor, torch.tensor]:
    
    """
    Get top most similar columns' embeddings to query using cosine similarity.
    """
    encoder = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", device='cpu')
    query_emb = encoder.encode(query, convert_to_tensor=True, device='cpu').unsqueeze(0)

    similarity_scores = F.cosine_similarity(query_emb, all_embs)
    top_results = torch.nonzero(similarity_scores > threshold).squeeze()

    # if top_results is empty, return empty tensors
    if top_results.numel() == 0:
        return torch.tensor([]), torch.tensor([])

    # if only 1 result is returned, we need to convert it to a tensor
    elif top_results.numel() == 1:
        return torch.tensor([similarity_scores[top_results]]), torch.tensor([top_results])
    else:
        top_k_scores, top_k_indices = torch.topk(similarity_scores[top_results], k=min(top_k, top_results.numel()))
        return top_k_scores, top_results[top_k_indices]
    
    
    
def format_topk_sql(topk_table_columns: dict[str, list[tuple[str, str, str]]], shuffle: bool) -> str:
    if len(topk_table_columns) == 0:
        return ""

    md_str = "\n"
    # shuffle the keys in topk_table_columns
    table_names = list(topk_table_columns.keys())
    if shuffle:
        np.random.seed(0)
        np.random.shuffle(table_names)
    for table_name in table_names:
        columns_str = ""
        columns = topk_table_columns[table_name]
        if shuffle:
            np.random.seed(0)
            np.random.shuffle(columns)
        for column_tuple in columns:
            if len(column_tuple) > 2:
                columns_str += (
                    f"\n  {column_tuple[0]} {column_tuple[1]}, --{column_tuple[2]}"
                )
            else:
                columns_str += f"\n  {column_tuple[0]} {column_tuple[1]}, "
        md_str += f"CREATE TABLE {table_name} ({columns_str}\n);\n"
    md_str += "\n"
    return md_str

In [30]:
question = 'Fetch the count of male and female in the data.'
EMBEDDING_PATH = 'embs'
TOP_K_LIMIT = 25 # number of columns to include in the prompt
PRUNE_LIMIT = 5 # minimum number of columns above which a given DDL will be pruned
num_cols = 0
TAB_DETAILS = []


for col in sqlglot.parse_one(schema, dialect='snowflake').find_all(sqlglot.exp.ColumnDef):
    num_cols += 1
    TAB_DETAILS.append([table_name, col.alias_or_name, col.find(sqlglot.exp.DataType).__str__(), col.find(sqlglot.exp.ColumnConstraint)])
        
print(TAB_DETAILS)


encoder = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", device='cpu')

column_descriptions = []
column_descriptions_typed = []

for row in TAB_DETAILS:
    tab_name, col_name, col_dtype, col_desc = row

    col_str = f"{tab_name}.{col_name}:{col_desc}"
    col_str_typed = f"{tab_name}.{col_name},{col_dtype},{col_desc}"

    column_descriptions.append(col_str)
    column_descriptions_typed.append(col_str_typed)
column_embs = encoder.encode(column_descriptions, convert_to_tensor=True, device='cpu')


# 1a) get top k columns
top_k_scores, top_k_indices = knn_(question, column_embs, top_k=5, threshold=0.0)
topk_table_columns = {}
table_column_names = set()

for score, index in zip(top_k_scores, top_k_indices):
    table_name, column_info = column_descriptions_typed[index].split(".", 1)
    column_tuple = re.split(r',\s*(?![^()]*\))', column_info, maxsplit=2) #split only on commas outside parantheses
    if table_name not in topk_table_columns:
        topk_table_columns[table_name] = []
    topk_table_columns[table_name].append(column_tuple)
    table_column_names.add(f"{table_name}.{column_tuple[0]}")
    # print("INCLUDED by embs: ", column_tuple)

# 1b) get columns which match terms in question
nlp = spacy.load("en_core_web_trf")
question_doc = nlp(question)
q_filtered_tokens = [token.lemma_.lower() for token in question_doc if not token.is_stop]
q_alpha_tokens = [i for i in q_filtered_tokens if (len(i)>1 and i.isalpha())]


TIME_TERMS = ['when', 'time', 'hour', 'minute', 'second', 
            'day', 'yesterday', 'today', 'tomorrow', 
            'week', 'month', 'year', 
            'duration', 'date']

time_in_q = False

nlp_ner = spacy.load("en_core_web_md")
q_ner_doc = nlp_ner(question)
ent_types = [w.label_ for w in q_ner_doc.ents]

if 'DATE' in ent_types or 'TIME' in ent_types:
    time_in_q = True
elif any([term in question.lower() for term in TIME_TERMS]):
    time_in_q = True
elif set(q_alpha_tokens).intersection(set(TIME_TERMS)):
    time_in_q = True

for col_details in column_descriptions_typed:
    table_name, column_info = col_details.split(".", 1)
    column_tuple = re.split(r',\s*(?![^()]*\))', column_info, maxsplit=2) #split only on commas outside parantheses
    col_name = column_tuple[0]

    if column_tuple in topk_table_columns[table_name]:
        # print("SKIPPING: ", column_tuple)
        continue

    # if question concerns time, add time-related columns
    if time_in_q and any([timetype in column_tuple[1] for timetype in ['DATE', 'TIMESTAMP']]):
        if table_name not in topk_table_columns:
            topk_table_columns[table_name] = []
        if column_tuple not in topk_table_columns[table_name]:
            topk_table_columns[table_name].append(column_tuple)
        table_column_names.add(f"{table_name}.{column_tuple[0]}")
        continue

    # if question-token-lemmas overlap with column-token-lemmas, add the column
    column_doc = nlp(col_name.replace('_', ' '))
    col_tokens = [token.lemma_.lower() for token in column_doc if not token.is_stop]
    col_alpha_tokens = [i for i in col_tokens if (len(i)>1 and i.isalpha())]
    if set(col_alpha_tokens).intersection(set(q_alpha_tokens)):
        if table_name not in topk_table_columns:
            topk_table_columns[table_name] = []
        if column_tuple not in topk_table_columns[table_name]:
            topk_table_columns[table_name].append(column_tuple)
        table_column_names.add(f"{table_name}.{column_tuple[0]}")

# 4) format metadata string
pruned_schema = format_topk_sql(topk_table_columns, shuffle=False)
print(pruned_schema)

[['Churn_Modelling', 'RowNumber', 'BIGINT', None], ['Churn_Modelling', 'CustomerId', 'BIGINT', None], ['Churn_Modelling', 'Surname', 'TEXT', None], ['Churn_Modelling', 'CreditScore', 'BIGINT', None], ['Churn_Modelling', 'Geography', 'TEXT', None], ['Churn_Modelling', 'Gender', 'TEXT', None], ['Churn_Modelling', 'Age', 'BIGINT', None], ['Churn_Modelling', 'Tenure', 'BIGINT', None], ['Churn_Modelling', 'Balance', 'FLOAT', None], ['Churn_Modelling', 'NumOfProducts', 'BIGINT', None], ['Churn_Modelling', 'HasCrCard', 'BIGINT', None], ['Churn_Modelling', 'IsActiveMember', 'BIGINT', None], ['Churn_Modelling', 'EstimatedSalary', 'FLOAT', None], ['Churn_Modelling', 'Exited', 'BIGINT', None]]

CREATE TABLE Churn_Modelling (
  Gender TEXT, --None
  RowNumber BIGINT, --None
  EstimatedSalary FLOAT, --None
  Age BIGINT, --None
  Surname TEXT, --None
);




In [31]:
prompt_template = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION] 

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'

## Database Schema 
This query will run on a database whose schema is represented in this string: {db_schema} 

### Answer 
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION] [SQL]
"""

prompt = prompt_template.format(question=question, db_schema = pruned_schema)
print(prompt)

### Task
Generate a SQL query to answer [QUESTION]Fetch the count of male and female in the data.[/QUESTION] 

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'

## Database Schema 
This query will run on a database whose schema is represented in this string: 
CREATE TABLE Churn_Modelling (
  Gender TEXT, --None
  RowNumber BIGINT, --None
  EstimatedSalary FLOAT, --None
  Age BIGINT, --None
  Surname TEXT, --None
);

 

### Answer 
Given the database schema, here is the SQL query that answers [QUESTION]Fetch the count of male and female in the data.[/QUESTION] [SQL]



### Prompt the LLM 

We will use Ollama (running locally or over the network) to prompt our LLM of choice (in this case, we are using `sqlglot-7b-2`)

In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [33]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# else, load in 8 bits – this is a bit slower
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    # torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map="auto",
    use_cache=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [34]:
import sqlparse

def generate_query(quest):
    updated_prompt = prompt.format(question=quest)
    print(updated_prompt)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [35]:
generated_sql = generate_query(question)

### Task
Generate a SQL query to answer [QUESTION]Fetch the count of male and female in the data.[/QUESTION] 

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'

## Database Schema 
This query will run on a database whose schema is represented in this string: 
CREATE TABLE Churn_Modelling (
  Gender TEXT, --None
  RowNumber BIGINT, --None
  EstimatedSalary FLOAT, --None
  Age BIGINT, --None
  Surname TEXT, --None
);

 

### Answer 
Given the database schema, here is the SQL query that answers [QUESTION]Fetch the count of male and female in the data.[/QUESTION] [SQL]



In [37]:
engine = create_engine(f"sqlite:///mysqlitedb.db")
with engine.connect() as conn, conn.begin():
    query_result = pd.read_sql_query(generated_sql, conn)

query_result

,Gender,gender_count
0,Female,4543
1,Male,5457
